In [1]:
import math
from tqdm import tqdm
import scvi
import torch
import numpy as np
import pytorch_lightning as pl
import os

import scanpy as sc

from pytorch_lightning.callbacks import TQDMProgressBar
from torchcfm.conditional_flow_matching import *
import scanpy as sc
from datamodules import  SCFMDataset, cfm_collate, StratifiedBatchSampler, ot_collate
from torch.utils.data import RandomSampler
from sc_etl_utils import *
from arch import *
import json

import scvi
import torch
import numpy as np
import pytorch_lightning as pl
import os

import scanpy as sc

from torchcfm.conditional_flow_matching import *
import scanpy as sc
import hashlib
from llm import MAE

In [2]:
# load some data
adata = sc.read_h5ad('/orcd/archive/abugoot/001/Projects/dlesman/datasets/satija_IFNB_HVG_and_perturbed_genes_raw.h5ad')

In [3]:
gene_map = {k: i for i, k in enumerate(adata.var.index)}
gene_map = gene_map | {'NT': max(gene_map.values()) + 1}
gene_unmap = {gene_map[k]: k for k in gene_map}
perts = adata.obs.gene.unique().map(gene_map)
adata.obs['pert_type'] = adata.obs.gene.map(gene_map)
pert_ids = np.array(adata.obs['pert_type'])
pert_mat = np.arange(pert_ids.max() + 1)[:, None]

In [4]:
cell_col = 'cell_type'
pert_col = 'pert_type'
control_pert, holdout_cells, holdout_perts = gene_map['NT'], ['HT29'], [gene_map['USP18']]

In [5]:
from sc_etl_utils import *
control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx = get_train_eval_idxs(
    adata, control_pert, holdout_cells, holdout_perts, cell_col=cell_col, pert_col=pert_col
)

_, _, cell_types = get_identity_features(
    adata, cell_col=cell_col, pert_col=pert_col, cell_type_features=False
)

adata.obsm["standard"] = adata.X
X = adata.obsm["standard"]
X = X.toarray()
X = X / X.sum(axis=1)[:, None]
X = np.log(X * 10_000. + 1)

control_train, pert_train, pert_ids_train, control_cell_types, pert_cell_types, control_eval, pert_eval, pert_ids_eval = get_train_eval(
    X, pert_ids, cell_types, control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx
)


Controls: 14582, Perturbations: 313080,  Eval: 880


In [6]:
train = np.vstack([control_train, pert_train])

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Create a custom Dataset class
class NumpyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return torch.from_numpy(self.data[index]).float()

    def __len__(self):
        return len(self.data)

# Create an instance of the dataset
dataset = NumpyDataset(train)

In [7]:
batch_size = 512
dset = SCFMDataset(
    control_train, pert_train, 
    pert_ids_train, pert_mat, 
    control_cell_types, pert_cell_types,
    batch_size=batch_size, size=X.shape[0]
)
ns = np.array([[t.shape[0] for t in ts] for ts in dset.target])
dl = torch.utils.data.DataLoader(
    dset, collate_fn=ot_collate, 
    batch_sampler=StratifiedBatchSampler(
        ns=ns, batch_size=512
    )
)

Strata probs [0.         0.0001086  0.00030344 0.00039606 0.00040565 0.00041523
 0.00042162 0.00043439 0.00045356 0.00045995 0.00049189 0.00052063
 0.00054938 0.00056535 0.00061007 0.00062284 0.00062604 0.00062604
 0.00063562 0.00068673 0.00075061 0.00076977 0.00077297 0.00077935
 0.00079852 0.0008081  0.00084323 0.0008624  0.00087837 0.00088156
 0.00088795 0.00088795 0.00089753 0.00089753 0.0009167  0.00092628
 0.00093906 0.00094864 0.00099016 0.00099016 0.00100294 0.00104127
 0.00104446 0.00105085 0.00109237 0.00110834 0.00110834 0.00111154
 0.00111473 0.00111473 0.0011307  0.0011339  0.00113709 0.00116584
 0.00117222 0.00117222 0.00121375 0.00121694 0.00122333 0.00122652
 0.00124569 0.00124888 0.00124888 0.00126485 0.00128082 0.00129999
 0.00130318 0.00130318 0.00131596 0.00131596 0.00132554 0.00133193
 0.00133512 0.0013447  0.00135109 0.00137026 0.00137984 0.00137984
 0.00139262 0.0014022  0.00140539 0.00140539 0.00141178 0.00141497
 0.00141817 0.00143733 0.00146288 0.00146608 0.00

In [8]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np


class GeneEmbedding(torch.nn.Module):
    def __init__(self, input_dim, emb_dim=128):
        super().__init__()
        self.pos = torch.nn.Parameter(torch.zeros(1, input_dim, emb_dim))
        nn.init.normal_(self.pos)
        
        
class PertEmbedder(torch.nn.Module):
    def __init__(self, gene_embedding):
        super().__init__()
        _, input_dim, emb_dim = gene_embedding.pos.shape
        self.gene_embedding = gene_embedding
        self.pert_token = torch.nn.Parameter(torch.zeros(emb_dim))
        nn.init.normal_(self.pert_token)
        
    def forward(self, pert_index, pert_expression):
        pert_pos = self.gene_embedding.pos[:, pert_index][0]

        pert_embed_and_expr = torch.cat(
            (
                pert_pos + self.pert_token, 
                pert_expression.unsqueeze(-1)
            ), dim=-1
        )
        return pert_embed_and_expr.unsqueeze(1)
    
class CellEncoder(torch.nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim, dropout_rate=0.1):
        super().__init__()
        self.input_dim = input_dim  
        self.latent_dim = latent_dim  # Dimension of the latent space
        self.dropout_rate = dropout_rate  
        

        # Encoder network definitions
        self.encoder_fc1 = nn.Linear(input_dim, hidden_dim, bias=False) 
        self.encoder_bn1 = nn.BatchNorm1d(hidden_dim)  
        self.encoder_fc2 = nn.Linear(hidden_dim, hidden_dim, bias=False) 
        self.encoder_bn2 = nn.BatchNorm1d(hidden_dim) 
        self.fc_mu = nn.Linear(hidden_dim, latent_dim, bias=True) 
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim, bias=True) 

    # Encoder function
    def forward(self, x):
        h = F.leaky_relu(self.encoder_bn1(self.encoder_fc1(x)))  
        h = F.dropout(h, p=self.dropout_rate, training=self.training)  
        h = F.leaky_relu(self.encoder_bn2(self.encoder_fc2(h)))  
        return nn.ELU()(self.fc_mu(h))# , self.fc_logvar(h)  


class ExprPred(torch.nn.Module):
    def __init__(self,
                 gene_embedding,
                 ff_dim=128,
                 ) -> None:
        super().__init__()
        
        _, _, emb_dim = gene_embedding.pos.shape
        self.gene_embedding = gene_embedding

        self.pred_expr = torch.nn.Sequential(
            torch.nn.Linear(emb_dim + emb_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, 1),
            torch.nn.ELU()
        ) 
        
        self.pred_bin = torch.nn.Sequential(
            torch.nn.Linear(emb_dim + emb_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, ff_dim),
            torch.nn.SELU(),
            torch.nn.Linear(ff_dim, 1),
            torch.nn.Sigmoid()
        ) 

    def forward(self, cell_embedding, pred_idx):
        embed_and_cell_embed = torch.cat(
            (
            torch.tile(cell_embedding.unsqueeze(1), (1, idx.shape[0], 1)),
            torch.tile(self.gene_embedding.pos[:, pred_idx], (cell_embedding.shape[0], 1, 1))
            ), dim=-1
        )
        pred_expr = self.pred_expr(embed_and_cell_embed) + 1
        pred_bin = self.pred_bin(embed_and_cell_embed)
        
        return pred_expr.squeeze(), pred_bin.squeeze()

class CMLP(pl.LightningModule):
    def __init__(self, feat_dim, cond_dim, out_dim=None, w1=128, w2=128, n_combo_layer=4, n_cond_layer=3, time_varying=False):
        super().__init__()
        self.time_varying = time_varying
        if out_dim is None:
            out_dim = feat_dim
        self.combo_net = torch.nn.Sequential(
            torch.nn.Linear(feat_dim + (1 if time_varying else 0) + cond_dim, w1), torch.nn.SELU(),
            *([torch.nn.Linear(w1, w1), torch.nn.SELU()] * n_combo_layer),
            torch.nn.Linear(w1, out_dim)
        )
        self.cond = None
        
    def forward(self, x, cond=None):
        if cond is None:
            cond = self.cond
        # cond = self.cond_net(cond)
        return self.combo_net(torch.cat([x, cond], dim=-1))

    
class BernoulliSampleLayer(nn.Module):
    def __init__(self):
        super(BernoulliSampleLayer, self).__init__()

    def forward(self, probs):
        sample = torch.bernoulli(probs)
        return sample + probs - probs.detach()    

    
class MAE(torch.nn.Module):
    def __init__(self,
                 input_dim,
                 ff_dim=128,
                 emb_dim=128,
                 encoder_layer=6,
                 ) -> None:
        super().__init__()
        
        self.gene_embedding = GeneEmbedding(input_dim=input_dim, emb_dim=emb_dim)
        self.pert_embedding = PertEmbedder(self.gene_embedding)
        self.encoder = CellEncoder(
            input_dim, emb_dim, hidden_dim=ff_dim
        )
        self.recon = ExprPred(self.gene_embedding)
        self.sparse_sampler = BernoulliSampleLayer()
        
        self.flow = CMLP(feat_dim=emb_dim, cond_dim=emb_dim, time_varying=True, w1=ff_dim)
        self.cfm_sampler = ExactOptimalTransportConditionalFlowMatcher(sigma=0.1)

    def forward(self, expr):
        cell_embedding = self.encoder(expr)
        # cell_embedding = features.mean(axis=1)
        return cell_embedding #, pred_expr.squeeze(), pred_bin.squeeze()
    
    def sparsify(self, pred_expr, pred_bin):
        sparsity = self.sparse_sampler(pred_bin)
        pred_expr *= sparsity
        return pred_expr
    
    def ae_loss(self, batch_emb, batch, gene_ids, lambd=0.5, return_recon=False):
        batch_bin = (batch > 0).float()
        batch_recon, batch_bin_recon = self.recon(batch_emb, gene_ids)
        recon_loss = torch.mean(batch_bin * (batch_recon - batch[:, gene_ids])**2) # / minibatch_size
        bin_loss = F.binary_cross_entropy(batch_bin_recon, batch_bin[:, gene_ids])
        loss = lambd * recon_loss + (1 - lambd) * bin_loss
        if return_recon:
            return loss, batch_recon, batch_bin_recon
        return loss
    
    def flow_loss(self, source_emb, target_emb, cond):
        t, xt, ut = self.cfm_sampler.sample_location_and_conditional_flow(
            source_emb, target_emb
        )

        inp = torch.cat([xt, t[:, None]], dim=-1)
        vt = model.flow(inp, cond)
        return torch.nn.functional.mse_loss(vt, ut) 


In [9]:
model = MAE(
    X.shape[1], 
    emb_dim=256, 
    encoder_layer=4,
    ff_dim=256
)
# model = torch.load(f"llm/v9")
device = 'cuda'
# device = 'cpu'
model = model.to(device)

In [10]:
base_learning_rate = 2e-4
weight_decay=0.0
total_epoch = 1000
warmup_epoch = 5
save_dir = "ae/v"
optim = torch.optim.Adam(model.parameters(), lr=base_learning_rate)
lr_func = lambda epoch: min((epoch + 1) / (warmup_epoch + 1e-5), 0.5 * (math.cos(epoch / total_epoch * math.pi) + 1))
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=lr_func, verbose=True)

Adjusting learning rate of group 0 to 4.0000e-05.


In [11]:
use_sparsity_loss = False
use_mask_task = True
use_active_weights = False
lr_step = 32
minibatch_size = 128

step_count = 0
optim.zero_grad()
pert_task = 0
idx = torch.arange(train.shape[1]).to(device)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=minibatch_size, shuffle=True)

for e in range(100):
    model.train()
    losses = {'loss': [], 'control': [], 'pert': [], 'flow': []}
    for batch in (pbar := tqdm(iter(dataloader))):
        step_count += 1
        batch = batch.to(device)
        idx = torch.arange(batch.shape[1]).to(device)
        batch_emb = model(batch)
        loss, batch_recon, batch_bin_recon = model.ae_loss(batch_emb, batch, idx, return_recon=True)
        loss.backward()
        optim.step()
        optim.zero_grad()
        with torch.no_grad():
            batch_recon = model.sparsify(batch_recon, batch_bin_recon)
            recon_loss = torch.mean((batch_recon - batch)**2)
        losses['loss'].append(recon_loss.item())
        if step_count % lr_step == 0:
            lr_scheduler.step()
        pbar.set_description(
            f"loss: {np.array(losses['loss'])[-lr_step:].mean():.3f}"
        )
    
    avg_loss = sum(losses['loss']) / len(losses['loss'])
    torch.save(model, f"{save_dir}{e}")
    # writer.add_scalar('mae_loss', avg_loss, global_step=e)
    print(f'In epoch {e}, average traning loss is {avg_loss}.')

loss: 1.671:   1%|██▌                                                                                                                                                                                    | 35/2560 [00:02<02:44, 15.31it/s]

Adjusting learning rate of group 0 to 8.0000e-05.


loss: 2.296:   3%|████▊                                                                                                                                                                                  | 67/2560 [00:04<02:42, 15.33it/s]

Adjusting learning rate of group 0 to 1.2000e-04.


loss: 2.204:   4%|███████                                                                                                                                                                                | 99/2560 [00:06<02:40, 15.32it/s]

...

loss: 0.951:  53%|███████████████████████████████████████████████████████████████████████████████████████████████▏                                                                                     | 1346/2560 [01:30<01:22, 14.78it/s]

Adjusting learning rate of group 0 to 1.8053e-04.


loss: 0.950:  54%|█████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                   | 1377/2560 [01:33<01:20, 14.78it/s]

Adjusting learning rate of group 0 to 1.8034e-04.


KeyboardInterrupt: 

In [ ]:
del pert_index
del ctrl 
del pert
del idx
del ctrl_emb
del ctrl_recon
del ctrl_bin_recon
del pert_emb
del pert_recon
del pert_bin_recon

In [12]:
optim.zero_grad()
import gc
torch.cuda.empty_cache()
gc.collect()

498

In [13]:
use_sparsity_loss = False
use_mask_task = True
use_active_weights = False
lr_step = 32
minibatch_size = 172

step_count = 0
optim.zero_grad()
pert_task = 0
for e in range(20):
    model.train()
    losses = {'loss': [], 'control': [], 'pert': [], 'flow': []}
    for (bcontrol, bpert, bpert_index) in (pbar := tqdm(iter(dl))):
        bcontrol, bpert, bpert_index = bcontrol.squeeze(), bpert.squeeze(), bpert_index.reshape(-1, 1)# , # bpert_expr.squeeze()
        curr_batch_size = bcontrol.shape[0]
        for i in range(curr_batch_size // minibatch_size):
            ctrl = bcontrol[(i * minibatch_size):((i + 1) * minibatch_size)]
            if ctrl.shape[0] == 0:
                continue
            pert = bpert[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = bpert_index[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = pert_index.squeeze()
            
            ctrl = ctrl.float().to(device)
            pert = pert.float().to(device)
            
            idx = torch.arange(ctrl.shape[1]).to(device)
            
            step_count += 1
            
            ctrl_emb = model(ctrl)
            ctrl_loss, ctrl_recon, ctrl_bin_recon = model.ae_loss(ctrl_emb, ctrl, idx, return_recon=True)
            
            pert_emb = model(pert)
            pert_loss, pert_recon, pert_bin_recon = model.ae_loss(pert_emb, pert, idx, return_recon=True)
            
            cond = model.gene_embedding.pos[:, pert_index][0]
            flow_loss = model.flow_loss(ctrl_emb, pert_emb, cond)
            
            loss = ctrl_loss + pert_loss + flow_loss
            loss.backward()
            optim.step()
            optim.zero_grad()
            losses['loss'].append(loss.item())
            losses['control'].append(ctrl_loss.item())
            losses['pert'].append(pert_loss.item())
            losses['flow'].append(flow_loss.item())
            if step_count % lr_step == 0:
                lr_scheduler.step()
            pbar.set_description(
                f"loss: {np.array(losses['loss'])[-lr_step:].mean():.3f}, tv: {np.array(losses['control'])[-lr_step:].mean():.3f}, ptv: {np.array(losses['pert'])[-lr_step:].mean():.3f}, flow: {np.array(losses['flow'])[-lr_step:].mean():.3f}"
            )
    
    avg_loss = sum(losses['control']) / len(losses['control'])
    torch.save(model, f"{save_dir}{e}")
    # writer.add_scalar('mae_loss', avg_loss, global_step=e)
    print(f'In epoch {e}, average traning loss is {avg_loss}.')

loss: 0.414, tv: 0.166, ptv: 0.170, flow: 0.078:   3%|███▉                                                                                                                                                | 16/611 [00:07<04:17,  2.31it/s]

Adjusting learning rate of group 0 to 1.8016e-04.


loss: 0.332, tv: 0.154, ptv: 0.159, flow: 0.019:   5%|███████▉                                                                                                                                            | 33/611 [00:14<04:06,  2.35it/s]

Adjusting learning rate of group 0 to 1.7997e-04.


loss: 0.336, tv: 0.159, ptv: 0.164, flow: 0.013:   8%|███████████▊                                                                                                                                        | 49/611 [00:21<04:05,  2.29it/s]

...

Adjusting learning rate of group 0 to 1.7921e-04.


loss: 0.290, tv: 0.140, ptv: 0.143, flow: 0.007:  19%|███████████████████████████▍                                                                                                                       | 114/611 [00:49<03:35,  2.31it/s]

Adjusting learning rate of group 0 to 1.7902e-04.


loss: 0.292, tv: 0.142, ptv: 0.144, flow: 0.006:  19%|████████████████████████████▏                                                                                                                      | 117/611 [00:50<03:34,  2.30it/s]


KeyboardInterrupt: 

In [19]:
import math
import torch
import numpy as np
from torchdyn.core import NeuralODE
from datamodules import torch_wrapper

def compute_conditional_flow(
    model, control, pert_ids, pert_mat, batch_size=100, num_steps=400, n_batches=1e8, true_bin=None
):
    node = NeuralODE(
        torch_wrapper(model.flow).to(device), solver="dopri5", sensitivity="adjoint"
    )
    n_samples = min(control.shape[0], pert_ids.shape[0])
    n_batches = min(math.ceil(n_samples / batch_size), n_batches)
    preds = np.zeros((min(n_batches * batch_size, n_samples), control.shape[1]))
    with torch.no_grad():
        for i in range(n_batches):
            control_batch = control[batch_size*i:min(batch_size*(i+1), n_samples)]
            pert_batch = pert_mat[pert_ids][batch_size*i:min(batch_size*(i+1), n_samples)]# [:control_batch.shape[0]]
            model.flow.cond = pert_batch.to(device)
            inp = control_batch.float()
            inp = inp.to(device)
            
            idx = torch.arange(control_eval.shape[1]).to(device)
            cell_embedding = model(inp)
            
            outp = node.trajectory(
                cell_embedding,
                t_span=torch.linspace(0, 1, num_steps)
            )
            outp = outp[-1, :, :]
            outp, outb = model.recon(cell_embedding, idx)
            if true_bin:
                outb = true_bin.to(device)
            outp = model.sparsify(outp, outb)
            outp = outp.cpu()
            preds[batch_size*i:batch_size*(i+1), :] = outp.squeeze()
            
    return preds

In [17]:
for cell_type, pert_type in zip(holdout_cells, holdout_perts):
    break

cell_type_names = adata.obs[cell_col]
pert_type_names = adata.obs[pert_col]
control_eval =  torch.tensor(X[(cell_type_names == cell_type) & (pert_type_names == gene_map['NT'])]).float()
true_pert = torch.tensor(X[(pert_type_names == pert_type) & (cell_type_names == cell_type)]).float()

In [20]:
pred = compute_conditional_flow(
    model,
    control_eval, 
    torch.tensor(np.repeat(pert_type, control_eval.shape[0])), 
    model.gene_embedding.pos[0]
)  

In [21]:
true_pert.mean(axis=0)

tensor([0.0490, 0.0000, 0.0000,  ..., 0.0000, 0.0484, 0.0105])

In [22]:
pred.mean(axis=0)

array([0.16726097, 0.00103138, 0.00203618, ..., 0.        , 0.04829858,
       0.10163789])

In [23]:
from os import listdir
from scipy.sparse import issparse
import anndata
import scanpy as sc
import numpy as np
import pandas as pd

from scipy.stats import pearsonr

import logging

logger = logging.getLogger(__name__)

def r2_mse_filename(pert, cell):
    return f'r2_and_mse_{pert}_{cell}.json'

def c_r_filename(pert, cell):
    return f'c_r_results_{pert}_{cell}.json'

def DEGs_overlap_filename(pert, cell):   
    return f'DEGs_overlaps_{pert}_{cell}.json'


def get_DEG_with_direction(gene, score):
    if score > 0:
        return(f'{gene}+')
    else:
        return(f'{gene}-')
        
def to_dense(X):
    if issparse(X):
        return X.toarray()
    else:
        return np.asarray(X)

def get_DEGs(control_adata, target_adata):
    temp_concat = anndata.concat([control_adata, target_adata], label = 'batch')
    sc.tl.rank_genes_groups(
        temp_concat, 'batch', method='wilcoxon', 
        groups = ['1'], ref = '0', rankby_abs = True, tie_correct=True
    )

    rankings = temp_concat.uns['rank_genes_groups']
    result_df = pd.DataFrame({'scores': rankings['scores']['1'],
                     'pvals_adj': rankings['pvals_adj']['1'],
                     'lfc': rankings['logfoldchanges']['1']},
                    index = rankings['names']['1'])
    return result_df

def get_eval(ctrl_adata, true_adata, pred_adata, DEGs, DEG_vals, pval_threshold, lfc_threshold):
        
    results_dict =  {}
    
    logger.debug(f"Computing R, R2, and MSE metrics")

    ctrl_mean = to_dense(ctrl_adata.X).mean(axis = 0)

    true_mean = to_dense(true_adata.X).mean(axis = 0)
    true_var = to_dense(true_adata.X).var(axis = 0)
    
    pred_mean = to_dense(pred_adata.X).mean(axis = 0)
    pred_var = to_dense(pred_adata.X).var(axis = 0)
    
    true_corr_mtx = np.corrcoef(to_dense(true_adata.X), rowvar=False).flatten()
    true_cov_mtx = np.cov(to_dense(true_adata.X), rowvar=False).flatten()
        
    pred_corr_mtx = np.corrcoef(to_dense(pred_adata.X), rowvar=False).flatten()
    pred_cov_mtx = np.cov(to_dense(pred_adata.X), rowvar=False).flatten()

    true_sub_diff = true_mean - ctrl_mean
    pred_sub_diff = pred_mean - ctrl_mean

    true_diff = true_mean/ctrl_mean
    pred_diff = pred_mean/ctrl_mean

    true_diff_mask = (np.isnan(true_diff) | np.isinf(true_diff))
    pred_diff_mask = (np.isnan(pred_diff) | np.isinf(pred_diff))
    
    common_mask = true_diff_mask | pred_diff_mask
    true_fold_diff = np.ma.array(true_diff, mask=common_mask).compressed()
    pred_fold_diff = np.ma.array(pred_diff, mask=common_mask).compressed()

    results_dict['all_genes_mean_sub_diff_R'] = pearsonr(true_sub_diff, pred_sub_diff)[0]
    results_dict['all_genes_mean_sub_diff_R2'] = pearsonr(true_sub_diff, pred_sub_diff)[0]**2
    results_dict['all_genes_mean_sub_diff_MSE'] = (np.square(true_sub_diff - pred_sub_diff)).mean(axis=0)

    results_dict['all_genes_mean_fold_diff_R'] = pearsonr(true_fold_diff, pred_fold_diff)[0]
    results_dict['all_genes_mean_fold_diff_R2'] = pearsonr(true_fold_diff, pred_fold_diff)[0]**2
    results_dict['all_genes_mean_fold_diff_MSE'] = (np.square(true_fold_diff - pred_fold_diff)).mean(axis=0)
    
    results_dict['all_genes_mean_R'] = pearsonr(true_mean, pred_mean)[0]
    results_dict['all_genes_mean_R2'] = pearsonr(true_mean, pred_mean)[0]**2
    results_dict['all_genes_mean_MSE'] = (np.square(true_mean - pred_mean)).mean(axis=0)

    results_dict['all_genes_var_R'] = pearsonr(true_var, pred_var)[0]
    results_dict['all_genes_var_R2'] = pearsonr(true_var, pred_var)[0]**2
    results_dict['all_genes_var_MSE'] = (np.square(true_var - pred_var)).mean(axis=0)
   
    corr_nas = np.logical_or(np.isnan(true_corr_mtx), np.isnan(pred_corr_mtx))
    cov_nas = np.logical_or(np.isnan(true_cov_mtx), np.isnan(pred_cov_mtx))

    results_dict['all_genes_corr_mtx_R'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]
    results_dict['all_genes_corr_mtx_R2'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]**2
    results_dict['all_genes_corr_mtx_MSE'] = (np.square(true_corr_mtx[~corr_nas] - pred_corr_mtx[~corr_nas])).mean(axis=0)

    results_dict['all_genes_cov_mtx_R'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]
    results_dict['all_genes_cov_mtx_R2'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]**2
    results_dict['all_genes_cov_mtx_MSE'] = (np.square(true_cov_mtx[~cov_nas] - pred_cov_mtx[~cov_nas])).mean(axis=0)

    if lfc_threshold:   
        significant_DEGs = DEGs[(DEGs['pvals_adj'] < pval_threshold) & (abs(DEGs) > lfc_threshold)]
    else:
        significant_DEGs = DEGs[DEGs['pvals_adj'] < pval_threshold]
    num_DEGs = len(significant_DEGs)
    DEG_vals.insert(0, num_DEGs)


    logger.debug(f"Significant DEGs {significant_DEGs}")
    
    for val in DEG_vals:

        logger.debug(f"Computing R, R2, and MSE metrics for top {val} DEGs")

        #If val == 1 we can't
        if ((val > num_DEGs) or (val == 0) or (val == 1)):
            results_dict[f'Top_{val}_DEGs_sub_diff_mean_R'] = None
            results_dict[f'Top_{val}_DEGs_sub_diff_mean_R2'] = None
            results_dict[f'Top_{val}_DEGs_sub_diff_mean_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_fold_diff_mean_R'] = None
            results_dict[f'Top_{val}_DEGs_fold_diff_mean_R2'] = None
            results_dict[f'Top_{val}_DEGs_fold_diff_mean_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_mean_R'] = None
            results_dict[f'Top_{val}_DEGs_mean_R2'] = None
            results_dict[f'Top_{val}_DEGs_mean_MSE'] = None

            results_dict[f'Top_{val}_DEGs_var_R'] = None
            results_dict[f'Top_{val}_DEGs_var_R2'] = None
            results_dict[f'Top_{val}_DEGs_var_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_corr_mtx_R'] = None
            results_dict[f'Top_{val}_DEGs_corr_mtx_R2'] = None
            results_dict[f'Top_{val}_DEGs_corr_mtx_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_cov_mtx_R'] = None
            results_dict[f'Top_{val}_DEGs_cov_mtx_R2'] = None
            results_dict[f'Top_{val}_DEGs_cov_mtx_MSE'] = None
        
        else:
            top_DEGs = significant_DEGs.iloc[0:val].index

            logger.debug(f"Top DEGs: {top_DEGs}")


            #Reshape --> If there is a single gene, the shape is (1,) and we need to reshape it to (1,1)

            ctrl_mean = to_dense(ctrl_adata[:,top_DEGs].X).mean(axis = 0)
            
            true_mean = to_dense(true_adata[:,top_DEGs].X).mean(axis = 0)

            logger.debug(f"Shape ctrl_adata with top DEGs: {ctrl_adata[:,top_DEGs].X.shape}, shape true_adata with top DEGs: {true_adata[:,top_DEGs].X.shape}")


            true_var = to_dense(true_adata[:,top_DEGs].X).var(axis = 0)
            true_corr_mtx = np.corrcoef(to_dense(true_adata[:,top_DEGs].X), rowvar=False).flatten()
            true_cov_mtx = np.cov(to_dense(true_adata[:,top_DEGs].X), rowvar=False).flatten()

            pred_mean = to_dense(pred_adata[:,top_DEGs].X).mean(axis = 0)
            logger.debug(f"Shape of true_mean shape: {true_mean.shape}, ctrl_mean shape: {ctrl_mean.shape}, pred_mean shape: {pred_mean.shape}")

            pred_var = to_dense(pred_adata[:,top_DEGs].X).var(axis = 0)
            pred_corr_mtx = np.corrcoef(to_dense(pred_adata[:,top_DEGs].X), rowvar=False).flatten()
            pred_cov_mtx = np.cov(to_dense(pred_adata[:,top_DEGs].X), rowvar=False).flatten()

            logger.debug(f"Shape of true_var shape: {true_var.shape}, pred_var shape: {pred_var.shape}")

            true_sub_diff = true_mean - ctrl_mean
            pred_sub_diff = pred_mean - ctrl_mean
        
            true_diff = true_mean/ctrl_mean
            pred_diff = pred_mean/ctrl_mean
        
            true_diff_mask = (np.isnan(true_diff) | np.isinf(true_diff))
            pred_diff_mask = (np.isnan(pred_diff) | np.isinf(pred_diff))
            
            common_mask = true_diff_mask | pred_diff_mask
            true_fold_diff = np.ma.array(true_diff, mask=common_mask).compressed()
            pred_fold_diff = np.ma.array(pred_diff, mask=common_mask).compressed()

            results_dict[f'Top_{val}_DEGs_sub_diff_R'] = pearsonr(true_sub_diff, pred_sub_diff)[0]
            results_dict[f'Top_{val}_DEGs_sub_diff_R2'] = pearsonr(true_sub_diff, pred_sub_diff)[0]**2
            results_dict[f'Top_{val}_DEGs_sub_diff_MSE'] = (np.square(true_sub_diff - pred_sub_diff)).mean(axis=0)
        
            results_dict[f'Top_{val}_DEGs_fold_diff_R'] = pearsonr(true_fold_diff, pred_fold_diff)[0]
            results_dict[f'Top_{val}_DEGs_fold_diff_R2'] = pearsonr(true_fold_diff, pred_fold_diff)[0]**2
            results_dict[f'Top_{val}_DEGs_fold_diff_MSE'] = (np.square(true_fold_diff - pred_fold_diff)).mean(axis=0)
    
            results_dict[f'Top_{val}_DEGs_mean_R'] = pearsonr(true_mean, pred_mean)[0]
            results_dict[f'Top_{val}_DEGs_mean_R2'] = pearsonr(true_mean, pred_mean)[0]**2
            results_dict[f'Top_{val}_DEGs_mean_MSE'] = (np.square(true_mean - pred_mean)).mean(axis=0)

            results_dict[f'Top_{val}_DEGs_var_R'] = pearsonr(true_var, pred_var)[0]
            results_dict[f'Top_{val}_DEGs_var_R2'] = pearsonr(true_var, pred_var)[0]**2
            results_dict[f'Top_{val}_DEGs_var_MSE'] = (np.square(true_var - pred_var)).mean(axis=0)
            
            corr_nas = np.logical_or(np.isnan(true_corr_mtx), np.isnan(pred_corr_mtx))
            cov_nas = np.logical_or(np.isnan(true_cov_mtx), np.isnan(pred_cov_mtx))

            results_dict[f'Top_{val}_DEGs_corr_mtx_R'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]
            results_dict[f'Top_{val}_DEGs_corr_mtx_R2'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]**2
            results_dict[f'Top_{val}_DEGs_corr_mtx_MSE'] = (np.square(true_corr_mtx[~corr_nas] - pred_corr_mtx[~corr_nas])).mean(axis=0)

            results_dict[f'Top_{val}_DEGs_cov_mtx_R'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]
            results_dict[f'Top_{val}_DEGs_cov_mtx_R2'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]**2
            results_dict[f'Top_{val}_DEGs_cov_mtx_MSE'] = (np.square(true_cov_mtx[~cov_nas] - pred_cov_mtx[~cov_nas])).mean(axis=0)

    return results_dict

def get_DEG_Coverage_Recall(true_DEGs, pred_DEGs, p_cutoff):
    sig_true_DEGs = true_DEGs[true_DEGs['pvals_adj'] < p_cutoff]
    true_DEGs_with_direction = [get_DEG_with_direction(gene,score) for gene, score in zip(sig_true_DEGs.index, sig_true_DEGs['scores'])]
    sig_pred_DEGs = pred_DEGs[pred_DEGs['pvals_adj'] < p_cutoff]
    pred_DEGs_with_direction = [get_DEG_with_direction(gene,score) for gene, score in zip(sig_pred_DEGs.index, sig_pred_DEGs['scores'])]
    num_true_DEGs = len(true_DEGs_with_direction)
    num_pred_DEGs = len(pred_DEGs_with_direction)
    num_overlapping_DEGs = len(set(true_DEGs_with_direction).intersection(set(pred_DEGs_with_direction)))
    if num_true_DEGs > 0: 
        COVERAGE = num_overlapping_DEGs/num_true_DEGs
    else:
        COVERAGE = None
    if num_pred_DEGs > 0:
        RECALL = num_overlapping_DEGs/num_pred_DEGs
    else:
        RECALL = None
    return COVERAGE, RECALL

def get_DEGs_overlaps(true_DEGs, pred_DEGs, DEG_vals, pval_threshold, lfc_threshold):
    if lfc_threshold:
        significant_true_DEGs = true_DEGs[(true_DEGs['pvals_adj'] < pval_threshold) & (abs(true_DEGs['lfc']) > lfc_threshold)]
        significant_pred_DEGs = pred_DEGs[(pred_DEGs['pvals_adj'] < pval_threshold) & (abs(pred_DEGs['lfc']) > lfc_threshold)]
    else:
        significant_true_DEGs = true_DEGs[true_DEGs['pvals_adj'] < pval_threshold]
        significant_pred_DEGs = pred_DEGs[pred_DEGs['pvals_adj'] < pval_threshold]

    true_DEGs_for_comparison = [get_DEG_with_direction(gene,score) for gene, score in zip(significant_true_DEGs.index, significant_true_DEGs['scores'])]   
    pred_DEGs_for_comparison = [get_DEG_with_direction(gene,score) for gene, score in zip(significant_pred_DEGs.index, significant_pred_DEGs['scores'])]
    
    logger.debug(f"Computing DEG overlaps, # of significant DEGs in true data: {len(true_DEGs_for_comparison)}, # of significant DEGs in pred data: {len(pred_DEGs_for_comparison)}")
    num_DEGs = len(significant_true_DEGs)
    DEG_vals.insert(0, num_DEGs)
    
    results = {}
    for val in DEG_vals:
        if val > num_DEGs:
            results[f'Overlap_in_top_{val}_DEGs'] = None
        else:
            results[f'Overlap_in_top_{val}_DEGs'] = len(set(true_DEGs_for_comparison[0:val]).intersection(set(pred_DEGs_for_comparison[0:val])))

    intersection = len(set(true_DEGs_for_comparison).intersection(set(pred_DEGs_for_comparison)))
    union = len(set(true_DEGs_for_comparison).union(set(pred_DEGs_for_comparison)))
    if union > 0:
        results['Jaccard'] = intersection/union
    else:
        results['Jaccard'] = None
    
    return results

In [24]:
import copy
adata_ = copy.deepcopy(adata)
adata_.X = X

In [25]:
adata_ctrl = adata_[(cell_type_names == cell_type) & (pert_type_names == gene_map['NT'])].copy()
adata_pert = adata_[(pert_type_names == pert_type) & (cell_type_names == cell_type)].copy()
adata_pert_pred = adata_ctrl.copy()
adata_pert_pred.X = pred

In [28]:
degs = get_DEGs(adata_ctrl, adata_pert)
degs_pred = get_DEGs(adata_ctrl, adata_pert_pred)

pdefault=0.05

significant_degs = degs[degs['pvals_adj'] < pdefault]
significant_degs_pred = degs_pred[degs_pred['pvals_adj'] < pdefault]

/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:322: RuntimeWarning: invalid value encountered in divide
  scores[group_index, :] = (
/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:322: RuntimeWarning: invalid value encountered in divide
  scores[group_index, :] = (


In [31]:
res = get_eval(adata_ctrl, adata_pert, adata_pert_pred, degs, [10, 25, 50, 100], pdefault, None)
print(res)

/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/tmp/ipykernel_854526/2125490035.py:73: RuntimeWarning: divide by zero encountered in divide
  true_diff = true_mean/ctrl_mean
/tmp/ipykernel_854526/2125490035.py:73: RuntimeWarning: invalid value encountered in divide
  true_diff = true_mean/ctrl_mean
/tmp/ipykernel_854526/2125490035.py:74: RuntimeWarning: divide by zero encountered in divide
  pred_diff = pred_mean/ctrl_mean
/tmp/ipykernel_854526/2125490035.py:74: RuntimeWarning: invalid value encountered in divide
  pred_diff = pred_mean/ctrl_mean


{'all_genes_mean_sub_diff_R': 0.05029209858854033, 'all_genes_mean_sub_diff_R2': 0.0025292951804394602, 'all_genes_mean_sub_diff_MSE': 0.36348496993582685, 'all_genes_mean_fold_diff_R': 0.16582671303629845, 'all_genes_mean_fold_diff_R2': 0.027498498756422875, 'all_genes_mean_fold_diff_MSE': 322.7634392561425, 'all_genes_mean_R': 0.6568438458438476, 'all_genes_mean_R2': 0.4314438378229362, 'all_genes_mean_MSE': 0.36348496993582685, 'all_genes_var_R': 0.6011030204631375, 'all_genes_var_R2': 0.36132484120990704, 'all_genes_var_MSE': 1.0966558748824586, 'all_genes_corr_mtx_R': 0.4622095705945211, 'all_genes_corr_mtx_R2': 0.21363768714917158, 'all_genes_corr_mtx_MSE': 0.0017465257027203752, 'all_genes_cov_mtx_R': 0.40612418943794526, 'all_genes_cov_mtx_R2': 0.16493685724662804, 'all_genes_cov_mtx_MSE': 0.0019094857114673165, 'Top_169_DEGs_sub_diff_R': 0.40730274960602525, 'Top_169_DEGs_sub_diff_R2': 0.1658955298366285, 'Top_169_DEGs_sub_diff_MSE': 1.0310371538274536, 'Top_169_DEGs_fold_diff

In [30]:
len(significant_degs.index[:10].intersection(significant_degs_pred.index[:10])), \
len(significant_degs.index[:25].intersection(significant_degs_pred.index[:25])), \
len(significant_degs.index[:50].intersection(significant_degs_pred.index[:50])), \
len(significant_degs.index[:100].intersection(significant_degs_pred.index[:100])), \
len(significant_degs.index.intersection(significant_degs_pred.index))

(0, 2, 6, 18, 159)